In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import csv

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import datetime
def dateparse (time_in_secs):    
    return datetime.datetime.fromtimestamp(float(time_in_secs))
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)

with open('train.csv') as csvfile:
	data = pd.read_csv(csvfile,delimiter=',',parse_dates=True,date_parser=dateparse) 
	fancy = data.corr()
	print(fancy)
	fancy.to_csv('correlation.csv')
	imp.fit([data['STARTING_LATITUDE']])
	imp.transform([data['STARTING_LATITUDE']])
	imp.fit([data['STARTING_LONGITUDE']])
	imp.transform([data['STARTING_LONGITUDE']])
	imp.fit([data['DESTINATION_LATITUDE']])
	imp.transform([data['DESTINATION_LATITUDE']])
	imp.fit([data['DESTINATION_LONGITUDE']])
	imp.transform([data['DESTINATION_LONGITUDE']])
	#data['STARTING_LATITUDE'].fillna(data['STARTING_LATITUDE'].interpolate(),inplace=True)
	data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'])
	data['TIMESTAMP'] = (data['TIMESTAMP'] - data['TIMESTAMP'].min())  / np.timedelta64(1,'D')
	
	#data['STARTING_LONGITUDE'].fillna(data['STARTING_LONGITUDE'].interpolate(),inplace=True)
	#data['DESTINATION_LATITUDE'].fillna(data['DESTINATION_LATITUDE'].interpolate(),inplace=True)
	#data['DESTINATION_LONGITUDE'].fillna(data['DESTINATION_LONGITUDE'].interpolate(),inplace=True)
	data['TOTAL_LUGGAGE_WEIGHT'].fillna(0.0,inplace=True)
	data['WAIT_TIME'].fillna(0.0,inplace=True)
	#data['value'].fillna(data['value'].interpolate(),inplace=True)
	#print(data.isnull().sum())
	
	lbl = preprocessing.LabelEncoder()
	lbl.fit(list(data['VEHICLE_TYPE'].values))
	data['VEHICLE_TYPE'] = lbl.transform(list(data['VEHICLE_TYPE'].values))
	#print(data.head())
	
	
	y = data['FARE']
	del data['FARE']
	del data['ID']
	X = data
	X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=1)
    
	from xgboost.sklearn import XGBRegressor
	params = {'max_depth' : [3,4,5,6,7,8,9,10],'min_child_weight': [1,2,3,4,5,6,7,8,9,10,11,12],'n_estimators':list(range(300, 1000)),'learning_rate':[0.05,0.1,0.15,0.2]}
	xcl = XGBRegressor(learning_rate=0.1)
	reg7 = GridSearchCV(xcl,params,n_jobs=3,cv=5)
	reg7.fit(X ,y)
	print(reg7.best_params_)
	print(reg7.best_score_)
	prediction7 = reg7.best_estimator_.predict(X_test)
	print("XGB",r2_score(y_test,prediction7))

                             ID  STARTING_LATITUDE  STARTING_LONGITUDE  \
ID                     1.000000           0.006497            0.007211   
STARTING_LATITUDE      0.006497           1.000000            0.188152   
STARTING_LONGITUDE     0.007211           0.188152            1.000000   
DESTINATION_LATITUDE   0.005248           0.999841            0.187969   
DESTINATION_LONGITUDE  0.005878           0.187988            0.999901   
TOTAL_LUGGAGE_WEIGHT   0.002161          -0.028586           -0.027185   
WAIT_TIME              0.001527           0.040090            0.009787   
TRAFFIC_STUCK_TIME     0.000992          -0.034019           -0.009962   
DISTANCE              -0.001623          -0.067880           -0.022807   
FARE                   0.015388           0.200948           -0.045350   

                       DESTINATION_LATITUDE  DESTINATION_LONGITUDE  \
ID                                 0.005248               0.005878   
STARTING_LATITUDE                  0.999841  

/home/vampire/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
